# Delta Lake Managed vs External Tables

This notebook demonstrates how to create managed and external tables with Delta Lake.

In [1]:
import pyspark
from delta import *
from pyspark.sql.types import *

In [2]:
builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

In [3]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/matthew.powers/opt/miniconda3/envs/pyspark-322-delta-200/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/matthew.powers/.ivy2/cache
The jars for the packages stored in: /Users/matthew.powers/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-80f235c6-bd6d-48c4-a853-f1a500a0dca5;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.0.0 in central
	found io.delta#delta-storage;2.0.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 347ms :: artifacts dl 22ms
	:: modules in use:
	io.delta#delta-core_2.12;2.0.0 from central in [default]
	io.delta#delta-storage;2.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number|

## Create external Delta Lake table with save

In [4]:
columns = ["movie", "release_date"]
data = [("The Godfather", 1972), ("Detective Pikachu", 2019), ("Donny Darko", 2001)]
rdd = spark.sparkContext.parallelize(data)
df = rdd.toDF(columns)

In [5]:
df.write.format("delta").save("external_table1")

In [6]:
spark.read.format("delta").load("external_table1").show()

+-----------------+------------+
|            movie|release_date|
+-----------------+------------+
|Detective Pikachu|        2019|
|    The Godfather|        1972|
|      Donny Darko|        2001|
+-----------------+------------+



In [7]:
%rm -rf external_table1

## Create external Delta Lake table with saveAsTable

In [8]:
df.write.format("delta").option("path", "external_table2").saveAsTable(
    "default.external_table2"
)

In [10]:
spark.sql("select * from external_table2").show()

AnalysisException: `default`.`external_table2` is not a Delta table.

In [15]:
%rm -rf external_table2

## Creating Delta Lake Managed Table

In [9]:
df.write.format("delta").saveAsTable("some_managed_table")

In [10]:
spark.sql("select * from some_managed_table").show()

+-----------------+------------+
|            movie|release_date|
+-----------------+------------+
|Detective Pikachu|        2019|
|    The Godfather|        1972|
|      Donny Darko|        2001|
+-----------------+------------+

